# GraphRAG Implementation with LlamaIndex - V2

[GraphRAG (Graphs + Retrieval Augmented Generation)](https://www.microsoft.com/en-us/research/project/graphrag/) combines the strengths of Retrieval Augmented Generation (RAG) and Query-Focused Summarization (QFS) to effectively handle complex queries over large text datasets. While RAG excels in fetching precise information, it struggles with broader queries that require thematic understanding, a challenge that QFS addresses but cannot scale well. GraphRAG integrates these approaches to offer responsive and thorough querying capabilities across extensive, diverse text corpora.

This notebook provides guidance on constructing the GraphRAG pipeline using the LlamaIndex PropertyGraph abstractions using Neo4J.

This notebook updates the GraphRAG pipeline to v2. If you haven’t checked v1 yet, you can find it [here](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/GraphRAG_v1.ipynb). Following are the updates to the existing implementation:

1. Integrate with Neo4J Graph database.
2. Embedding based retrieval.



## Installation

`graspologic` is used to use hierarchical_leiden for building communities.

In [ ]:
!pip install llama-index graspologic numpy==1.24.4 scipy==1.12.0 llama_index_graph_stores_neo4j

## Load Data

We will use a sample news article dataset retrieved from Diffbot, which Tomaz has conveniently made available on GitHub for easy access.

The dataset contains 2,500 samples; for ease of experimentation, we will use 50 of these samples, which include the `title` and `text` of news articles.

In [ ]:
!pip install llama-index-core

In [107]:
import pandas as pd

news = pd.read_csv(
    "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv"
)[:50]

news.head()

,title,date,text
0,Chevron: Best Of Breed,2031-04-06T01:36:32.000000000+00:00,JHVEPhoto Like many companies in the O&G secto...
1,FirstEnergy (NYSE:FE) Posts Earnings Results,2030-04-29T06:55:28.000000000+00:00,FirstEnergy (NYSE:FE – Get Rating) posted its ...
2,Dáil almost suspended after Sinn Féin TD put p...,2023-06-15T14:32:11.000000000+00:00,The Dáil was almost suspended on Thursday afte...
3,Epic’s latest tool can animate hyperrealistic ...,2023-06-15T14:00:00.000000000+00:00,"Today, Epic is releasing a new tool designed t..."
4,"EU to Ban Huawei, ZTE from Internal Commission...",2023-06-15T13:50:00.000000000+00:00,The European Commission is planning to ban equ...


Prepare documents as required by LlamaIndex

In [108]:
from llama_index.core import Document

documents = [
    Document(text=f"{row['title']}: {row['text']}")
    for i, row in news.iterrows()
]

## Setup API Key and LLM


We will use TGI to serve meta-llama/Meta-Llama-3-70B-Instruct on Gaudi 2 card.
```
volume=$PWD/data 
model=meta-llama/Meta-Llama-3-70B-Instruct
docker run -p 8080:80 -v $volume:/data --runtime=habana \
-e HABANA_VISIBLE_DEVICES=1 \
-e OMPI_MCA_btl_vader_single_copy_mechanism=none \
-e HUGGING_FACE_HUB_TOKEN=$hf_token \
--cap-add=sys_nice \
--ipc=host
ghcr.io/huggingface/tgi-gaudi:2.0.1 --model-id $model --max-input-tokens 3000 --max-total-tokens 8192
```

In [133]:
from llama_index.llms.text_generation_inference import TextGenerationInference

prompt = "once upon a time there was a pirate with a colorful personality"
llm = TextGenerationInference(
    model_url="http://127.0.0.1:8080",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature=0.7,
    max_tokens=5192,
    is_chat_model=False,
)
response = llm.complete(prompt=prompt)
print(response)

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/text_generation/client.py:70: DeprecationWarning: `text_generation` clients are deprecated and will be removed in the near future. Please use the `InferenceClient` from the `huggingface_hub` package instead.
  warnings.warn(DEPRECATION_WARNING, DeprecationWarning)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/text_generation/client.py:553: DeprecationWarning: `text_generation` clients are deprecated and will be removed in the near future. Please use the `InferenceClient` from the `huggingface_hub` package instead.
  warnings.warn(DEPRECATION_WARNING, DeprecationWarning)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('T

Shiver me timbers! Let me tell ye the tale of Captain Jack "Swashbucklin' Sam" Saunders, the most colorful pirate to ever sail the Seven Seas!

Captain Jack was a man of great charm and charisma, with a mop of curly red hair and a beard to match. His eyes sparkled like the stars on a clear night, and his booming laughter could be heard for miles. He wore a bright yellow coat with golden epaulets, a red bandana, and a pair of white boots that had seen their fair share of adventure.

Sam's ship, the "Maverick's Revenge," was a beauty, with three masts and a hull painted in stripes of blue, green, and yellow. The Jolly Roger flew high and proud, and the crew of scurvy dogs knew that with Captain Jack at the helm, they were in for a wild ride.

Sam's love of adventure and treasure was only matched by his love of music and song. He'd often break into a rousing sea shanty, his voice booming across the waves, and his crew would join in, their voices blending in perfect harmony. The "Maverick'

Alternatively, if you wanted to use a Huggingface TGI endpoint using the url you could use:
```
from llama_index.llms.text_generation_inference import TextGenerationInference
llm = TextGenerationInference(
    model_url="http://127.0.0.1:8080",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature=0.7,
    max_tokens=5192,
    is_chat_model=False,
)
```

## GraphRAGExtractor

The GraphRAGExtractor class is designed to extract triples (subject-relation-object) from text and enrich them by adding descriptions for entities and relationships to their properties using an LLM.

This functionality is similar to that of the `SimpleLLMPathExtractor`, but includes additional enhancements to handle entity, relationship descriptions. For guidance on implementation, you may look at similar existing [extractors](https://docs.llamaindex.ai/en/latest/examples/property_graph/Dynamic_KG_Extraction/?h=comparing).

Here's a breakdown of its functionality:

**Key Components:**

1. `llm:` The language model used for extraction.
2. `extract_prompt:` A prompt template used to guide the LLM in extracting information.
3. `parse_fn:` A function to parse the LLM's output into structured data.
4. `max_paths_per_chunk:` Limits the number of triples extracted per text chunk.
5. `num_workers:` For parallel processing of multiple text nodes.


**Main Methods:**

1. `__call__:` The entry point for processing a list of text nodes.
2. `acall:` An asynchronous version of __call__ for improved performance.
3. `_aextract:` The core method that processes each individual node.


**Extraction Process:**

For each input node (chunk of text):
1. It sends the text to the LLM along with the extraction prompt.
2. The LLM's response is parsed to extract entities, relationships, descriptions for entities and relations.
3. Entities are converted into EntityNode objects. Entity description is stored in metadata
4. Relationships are converted into Relation objects. Relationship description is stored in metadata.
5. These are added to the node's metadata under KG_NODES_KEY and KG_RELATIONS_KEY.

**NOTE:** In the current implementation, we are using only relationship descriptions. In the next implementation, we will utilize entity descriptions during the retrieval stage.

In [134]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

from typing import Any, List, Callable, Optional, Union, Dict
from IPython.display import Markdown, display

from llama_index.core.async_utils import run_jobs
from llama_index.core.indices.property_graph.utils import (
    default_parse_triplets_fn,
)
from llama_index.core.graph_stores.types import (
    EntityNode,
    KG_NODES_KEY,
    KG_RELATIONS_KEY,
    Relation,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.default_prompts import (
    DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
)
from llama_index.core.schema import TransformComponent, BaseNode
from llama_index.core.bridge.pydantic import BaseModel, Field


class GraphRAGExtractor(TransformComponent):
    """Extract triples from a graph.

    Uses an LLM and a simple prompt + output parsing to extract paths (i.e. triples) and entity, relation descriptions from text.

    Args:
        llm (LLM):
            The language model to use.
        extract_prompt (Union[str, PromptTemplate]):
            The prompt to use for extracting triples.
        parse_fn (callable):
            A function to parse the output of the language model.
        num_workers (int):
            The number of workers to use for parallel processing.
        max_paths_per_chunk (int):
            The maximum number of paths to extract per chunk.
    """

    llm: LLM
    extract_prompt: PromptTemplate
    parse_fn: Callable
    num_workers: int
    max_paths_per_chunk: int

    def __init__(
        self,
        llm: Optional[LLM] = None,
        extract_prompt: Optional[Union[str, PromptTemplate]] = None,
        parse_fn: Callable = default_parse_triplets_fn,
        max_paths_per_chunk: int = 10,
        num_workers: int = 4,
    ) -> None:
        """Init params."""
        from llama_index.core import Settings

        if isinstance(extract_prompt, str):
            extract_prompt = PromptTemplate(extract_prompt)

        super().__init__(
            llm=llm or Settings.llm,
            extract_prompt=extract_prompt or DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
            parse_fn=parse_fn,
            num_workers=num_workers,
            max_paths_per_chunk=max_paths_per_chunk,
        )

    @classmethod
    def class_name(cls) -> str:
        return "GraphExtractor"

    def __call__(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes."""
        return asyncio.run(
            self.acall(nodes, show_progress=show_progress, **kwargs)
        )

    async def _aextract(self, node: BaseNode) -> BaseNode:
        """Extract triples from a node."""
        assert hasattr(node, "text")

        text = node.get_content(metadata_mode="llm")
        try:
            llm_response = await self.llm.apredict(
                self.extract_prompt,
                text=text,
                max_knowledge_triplets=self.max_paths_per_chunk,
            )
            print(llm_response)
            entities, entities_relationship = self.parse_fn(llm_response)
        except ValueError:
            entities = []
            entities_relationship = []

        existing_nodes = node.metadata.pop(KG_NODES_KEY, [])
        existing_relations = node.metadata.pop(KG_RELATIONS_KEY, [])
        entity_metadata = node.metadata.copy()
        for entity, entity_type, description in entities:
            entity_metadata["entity_description"] = description
            entity_node = EntityNode(
                name=entity, label=entity_type, properties=entity_metadata
            )
            existing_nodes.append(entity_node)

        relation_metadata = node.metadata.copy()
        for triple in entities_relationship:
            subj, obj, rel, description = triple
            relation_metadata["relationship_description"] = description
            rel_node = Relation(
                label=rel,
                source_id=subj,
                target_id=obj,
                properties=relation_metadata,
            )

            existing_relations.append(rel_node)

        node.metadata[KG_NODES_KEY] = existing_nodes
        node.metadata[KG_RELATIONS_KEY] = existing_relations
        return node

    async def acall(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes async."""
        jobs = []
        for node in nodes:
            jobs.append(self._aextract(node))

        return await run_jobs(
            jobs,
            workers=self.num_workers,
            show_progress=show_progress,
            desc="Extracting paths from text",
        )

## GraphRAGStore

The `GraphRAGStore` class is an extension of the `Neo4jPropertyGraphStore`class, designed to implement GraphRAG pipeline. Here's a breakdown of its key components and functions:


The class uses community detection algorithms to group related nodes in the graph and then it generates summaries for each community using an LLM.


**Key Methods:**

`build_communities():`

1. Converts the internal graph representation to a NetworkX graph.

2. Applies the hierarchical Leiden algorithm for community detection.

3. Collects detailed information about each community.

4. Generates summaries for each community.

`generate_community_summary(text):`

1. Uses LLM to generate a summary of the relationships in a community.
2. The summary includes entity names and a synthesis of relationship descriptions.

`_create_nx_graph():`

1. Converts the internal graph representation to a NetworkX graph for community detection.

`_collect_community_info(nx_graph, clusters):`

1. Collects detailed information about each node based on its community.
2. Creates a string representation of each relationship within a community.

`_summarize_communities(community_info):`

1. Generates and stores summaries for each community using LLM.

`get_community_summaries():`

1. Returns the community summaries by building them if not already done.

In [148]:
import re
import networkx as nx
from graspologic.partition import hierarchical_leiden
from collections import defaultdict

from llama_index.core.llms import ChatMessage
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

from llama_index.llms.text_generation_inference import TextGenerationInference


class GraphRAGStore(Neo4jPropertyGraphStore):
    community_summary = {}
    entity_info = None
    max_cluster_size = 5

    def generate_community_summary(self, text):
        llm = TextGenerationInference(
            model_url="http://127.0.0.1:8080",
            model_name="meta-llama/Meta-Llama-3-8B-Instruct",
            temperature=0.7,
            max_tokens=5192,
            is_chat_model=False,
        )
        """Generate summary for a given text using an LLM."""
        messages = [
            ChatMessage(
                role="system",
                content=(
                    "You are provided with a set of relationships from a knowledge graph, each represented as "
                    "entity1->entity2->relation->relationship_description. Your task is to create a summary of these "
                    "relationships. The summary should include the names of the entities involved and a concise synthesis "
                    "of the relationship descriptions. The goal is to capture the most critical and relevant details that "
                    "highlight the nature and significance of each relationship. Ensure that the summary is coherent and "
                    "integrates the information in a way that emphasizes the key aspects of the relationships."
                ),
            ),
            ChatMessage(role="user", content=text),
        ]
        # response = OpenAI().chat(messages)
        response = llm.chat(messages)
        clean_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return clean_response

    def build_communities(self):
        """Builds communities from the graph and summarizes them."""
        nx_graph = self._create_nx_graph()
        community_hierarchical_clusters = hierarchical_leiden(
            nx_graph, max_cluster_size=self.max_cluster_size
        )
        self.entity_info, community_info = self._collect_community_info(
            nx_graph, community_hierarchical_clusters
        )
        self._summarize_communities(community_info)

    def _create_nx_graph(self):
        """Converts internal graph representation to NetworkX graph."""
        nx_graph = nx.Graph()
        triplets = self.get_triplets()
        for entity1, relation, entity2 in triplets:
            nx_graph.add_node(entity1.name)
            nx_graph.add_node(entity2.name)
            nx_graph.add_edge(
                relation.source_id,
                relation.target_id,
                relationship=relation.label,
                description=relation.properties["relationship_description"],
            )
        return nx_graph

    def _collect_community_info(self, nx_graph, clusters):
        """
        Collect information for each node based on their community,
        allowing entities to belong to multiple clusters.
        """
        entity_info = defaultdict(set)
        community_info = defaultdict(list)

        for item in clusters:
            node = item.node
            cluster_id = item.cluster

            # Update entity_info
            entity_info[node].add(cluster_id)

            for neighbor in nx_graph.neighbors(node):
                edge_data = nx_graph.get_edge_data(node, neighbor)
                if edge_data:
                    detail = f"{node} -> {neighbor} -> {edge_data['relationship']} -> {edge_data['description']}"
                    community_info[cluster_id].append(detail)

        # Convert sets to lists for easier serialization if needed
        entity_info = {k: list(v) for k, v in entity_info.items()}

        return dict(entity_info), dict(community_info)

    def _summarize_communities(self, community_info):
        """Generate and store summaries for each community."""
        for community_id, details in community_info.items():
            details_text = (
                "\n".join(details) + "."
            )  # Ensure it ends with a period
            self.community_summary[
                community_id
            ] = self.generate_community_summary(details_text)

    def get_community_summaries(self):
        """Returns the community summaries, building them if not already done."""
        if not self.community_summary:
            self.build_communities()
        return self.community_summary

## GraphRAGQueryEngine

The GraphRAGQueryEngine class is a custom query engine designed to process queries using the GraphRAG approach. It leverages the community summaries generated by the GraphRAGStore to answer user queries. Here's a breakdown of its functionality:

**Main Components:**

`graph_store:` An instance of GraphRAGStore, which contains the community summaries.
`llm:` A Language Model (LLM) used for generating and aggregating answers.


**Key Methods:**

`custom_query(query_str: str)`

1. This is the main entry point for processing a query. It retrieves community summaries, generates answers from each summary, and then aggregates these answers into a final response.

`generate_answer_from_summary(community_summary, query):`

1. Generates an answer for the query based on a single community summary.
Uses the LLM to interpret the community summary in the context of the query.

`aggregate_answers(community_answers):`

1. Combines individual answers from different communities into a coherent final response.
2. Uses the LLM to synthesize multiple perspectives into a single, concise answer.


**Query Processing Flow:**

1. Retrieve community summaries from the graph store.
2. For each community summary, generate a specific answer to the query.
3. Aggregate all community-specific answers into a final, coherent response.


**Example usage:**

```
query_engine = GraphRAGQueryEngine(graph_store=graph_store, llm=llm)

response = query_engine.query("query")
```

In [149]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.llms import LLM
from llama_index.core import PropertyGraphIndex

import re


class GraphRAGQueryEngine(CustomQueryEngine):
    graph_store: GraphRAGStore
    index: PropertyGraphIndex
    llm: LLM
    similarity_top_k: int = 20

    def custom_query(self, query_str: str) -> str:
        """Process all community summaries to generate answers to a specific query."""

        entities = self.get_entities(query_str, self.similarity_top_k)

        community_ids = self.retrieve_entity_communities(
            self.graph_store.entity_info, entities
        )
        community_summaries = self.graph_store.get_community_summaries()
        community_answers = [
            self.generate_answer_from_summary(community_summary, query_str)
            for id, community_summary in community_summaries.items()
            if id in community_ids
        ]

        final_answer = self.aggregate_answers(community_answers)
        return final_answer

    def get_entities(self, query_str, similarity_top_k):
        nodes_retrieved = index.as_retriever(
            similarity_top_k=similarity_top_k
        ).retrieve(query_str)

        enitites = set()
        pattern = r"(\w+(?:\s+\w+)*)\s*\({[^}]*}\)\s*->\s*([^(]+?)\s*\({[^}]*}\)\s*->\s*(\w+(?:\s+\w+)*)"

        for node in nodes_retrieved:
            matches = re.findall(pattern, node.text, re.DOTALL)

            for match in matches:
                subject = match[0]
                obj = match[2]
                enitites.add(subject)
                enitites.add(obj)

        return list(enitites)

    def retrieve_entity_communities(self, entity_info, entities):
        """
        Retrieve cluster information for given entities, allowing for multiple clusters per entity.

        Args:
        entity_info (dict): Dictionary mapping entities to their cluster IDs (list).
        entities (list): List of entity names to retrieve information for.

        Returns:
        List of community or cluster IDs to which an entity belongs.
        """
        community_ids = []

        for entity in entities:
            if entity in entity_info:
                community_ids.extend(entity_info[entity])

        return list(set(community_ids))

    def generate_answer_from_summary(self, community_summary, query):
        """Generate an answer from a community summary based on a given query using LLM."""
        prompt = (
            f"Given the community summary: {community_summary}, "
            f"how would you answer the following query? Query: {query}"
        )
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content="I need an answer based on the above information.",
            ),
        ]
        response = self.llm.chat(messages)
        cleaned_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return cleaned_response

    def aggregate_answers(self, community_answers):
        """Aggregate individual community answers into a final, coherent response."""
        # intermediate_text = " ".join(community_answers)
        prompt = "Combine the following intermediate answers into a final, concise response."
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content=f"Intermediate answers: {community_answers}",
            ),
        ]
        final_response = self.llm.chat(messages)
        cleaned_final_response = re.sub(
            r"^assistant:\s*", "", str(final_response)
        ).strip()
        return cleaned_final_response

##  Build End to End GraphRAG Pipeline

Now that we have defined all the necessary components, let’s construct the GraphRAG pipeline:

1. Create nodes/chunks from the text.
2. Build a PropertyGraphIndex using `GraphRAGExtractor` and `GraphRAGStore`.
3. Construct communities and generate a summary for each community using the graph built above.
4. Create a `GraphRAGQueryEngine` and begin querying.

### Create nodes/ chunks from the text.

In [137]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

In [138]:
len(nodes)

50

### Build ProperGraphIndex using `GraphRAGExtractor` and `GraphRAGStore`

The quality of the GraphRAG pipeline depends heavily on the quality of the generated Knowledge Graph. LLM is used to retreve entities, relationships and descriptions of those and those LLM generated outputs are parsed to find triplets and fill the graph store. You may need to tune the prompt to improve the answers and formatting.

In [139]:
KG_TRIPLET_EXTRACT_TMPL = """
-Goal-
Given a text document, identify all entities and their entity types from the text and all relationships among the identified entities.
Given the text, extract up to {max_knowledge_triplets} entity-relation triplets.

-Steps-
1. Identify all entities. For each identified entity, extract the following information:
- entity_name: Name of the entity, capitalized
- entity_type: Type of the entity
- entity_description: Comprehensive description of the entity's attributes and activities
Format each entity strictly as ("entity"$$$$<entity_name>$$$$<entity_type>$$$$<entity_description>). Pay attention to the dollar signs ($$$$) separating the fields and the parentheses surrounding the entire entity.
one example: ("entity"$$$$Apple$$$$Company$$$$Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services.)

2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relation: relationship between source_entity and target_entity
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other

Format each relationship strictly as ("relationship"$$$$<source_entity>$$$$<target_entity>$$$$<relation>$$$$<relationship_description>). Pay attention to the dollar signs ($$$$) separating the fields and the parentheses surrounding the entire entity.
one example: ("relationship"$$$$Apple$$$$Steve Jobs$$$$Founded$$$$Steve Jobs co-founded Apple Inc. in 1976.)

3. When finished, output.

-Real Data-
######################
text: {text}
######################
output:"""

In [151]:
# entity_pattern = r'\("entity"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'
# relationship_pattern = r'\("relationship"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'

entity_pattern = r'\("entity"\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\)'
relationship_pattern = (
    r'\("relationship"\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\)'
)


def parse_fn(response_str: str) -> Any:
    entities = re.findall(entity_pattern, response_str)
    relationships = re.findall(relationship_pattern, response_str)
    print("len of parsed entities", len(entities))
    print("len of parsed relationships", len(relationships))
    return entities, relationships


kg_extractor = GraphRAGExtractor(
    llm=llm,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL,
    max_paths_per_chunk=4,
    parse_fn=parse_fn,
)

## Docker Setup And Neo4J setup

To launch Neo4j locally, first ensure you have docker installed. Then, you can launch the database with the following docker command.

```
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```
From here, you can open the db at http://localhost:7474/. On this page, you will be asked to sign in. Use the default username/password of neo4j and neo4j.

Once you login for the first time, you will be asked to change the password. For convenience the below snippet of code allows you to change your password directly from code.

In [ ]:
from neo4j import GraphDatabase

# Connect to the Neo4j database
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"))


# Function to change the password
def change_password(tx, current_password, new_password):
    tx.run(
        "ALTER CURRENT USER SET PASSWORD FROM $current_password TO $new_password",
        current_password=current_password,
        new_password=new_password,
    )


# Execute the password change
with driver.session(database="system") as session:
    session.write_transaction(change_password, "neo4j", "your_neo4j_pswd")

# Close the driver
driver.close()

For convenience the below snippet of code allows you to change your password directly from code

In [14]:
from neo4j import GraphDatabase

# Connect to the Neo4j database
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"))


# Function to change the password
def change_password(tx, current_password, new_password):
    tx.run(
        "ALTER CURRENT USER SET PASSWORD FROM $current_password TO $new_password",
        current_password=current_password,
        new_password=new_password,
    )


# Execute the password change
with driver.session(database="system") as session:
    session.write_transaction(change_password, "neo4j", "neo4jrbb")

# Close the driver
driver.close()

/tmp/ipykernel_2674594/2724122089.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(change_password, "neo4j", "neo4jrbb")


In [152]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = GraphRAGStore(
    username="neo4j", password="neo4jrbb", url="bolt://localhost:7687"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [ ]:
We will use TEI to serve embedding model on Gaudi 2 card.

We will use TEI to serve BAAI/bge-large-en-v1.5 embedding model on Gaudi 2 card.
```
volume=$PWD/data  
model=BAAI/bge-large-en-v1.5
docker run -p 8081:80 -v $volume:/data --runtime=habana \
-e HABANA_VISIBLE_DEVICES=0 \
-e OMPI_MCA_btl_vader_single_copy_mechanism=none \
-e MAX_WARMUP_SEQUENCE_LENGTH=512 \
--cap-add=sys_nice \
--ipc=host
ghcr.io/huggingface/tei-gaudi:latest --model-id $model --pooling cls 
```

In [142]:
from llama_index.embeddings.text_embeddings_inference import (
    TextEmbeddingsInference,
)

embed_model = TextEmbeddingsInference(
    base_url="http://127.0.0.1:8081",
    model_name="BAAI/bge-large-en-v1.5",  # required for formatting inference text,
    timeout=60,  # timeout in seconds
    embed_batch_size=10,  # batch size for embedding
)

In [153]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex(
    nodes=nodes,
    llm=llm,
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    embed_model=embed_model,
    show_progress=True,
)

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; 

Based on the provided text, here is the output:

Entities:

("entity"$$$$Nirmal Bang$$$$Brokerage Firm$$$$Nirmal Bang is a financial services company that provides brokerage services to its clients.)

("entity"$$$$Tata Group$$$$Company$$$$Tata Group is an Indian multinational conglomerate company that operates in various sectors, including chemicals, steel, and automobiles.)

("entity"$$$$Tata Chemicals Ltd$$$$Company$$$$Tata Chemicals Ltd. is a chemical company that operates in the chemicals sector and is part of the Tata Group.)

("entity"$$$$Greynium Information Technologies$$$$Company$$$$Greynium Information Technologies is a technology company that provides information services to its clients.)

("entity"$$$$Stock Market$$$$Market$$$$The stock market is a platform where companies' shares are traded.)

("entity"$$$$Soda Ash$$$$Material$$$$Soda Ash is a chemical substance used in various industries, including glass manufacturing and detergents.)

("entity"$$$$Lithium Carbonate$$$$Ma

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)





Here is the output:

Entities:

("entity"$$$$Man City$$$$Team$$$$Manchester City Football Club is a professional football club based in Manchester, England, that plays in the Premier League.)

("entity"$$$$Arsenal$$$$Team$$$$Arsenal Football Club is a professional football club based in London, England, that plays in the Premier League.)

("entity"$$$$Brighton$$$$Team$$$$Brighton & Hove Albion Football Club is a professional football club based in Brighton, England, that plays in the Premier League.)

("entity"$$$$Manchester United$$$$Team$$$$Manchester United Football Club is a professional football club based in Manchester, England, that plays in the Premier League.)

("entity"$$$$Chelsea$$$$Team$$$$Chelsea Football Club is a professional football club based in London, England, that plays in the Premier League.)

("entity"$$$$Liverpool$$$$Team$$$$Liverpool Football Club is a professional football club based in Liverpool, England, that plays in the Premier League.)

("entity"$$$$Totte

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)





...

After analyzing the given text, I identified the following entities and relationships:

Entities:

1. ("entity"$$$$Ryanair$$$$Company$$$$Ryanair is an Irish low-cost airline that operates a network of short-haul and medium-haul routes in Europe and North Africa.)
2. ("entity"$$$$Aidan Murray$$$$Person$$$$Aidan Murray is a former chief pilot of Ryanair, who was sacked for alleged sexual harassment of female colleagues.)
3. ("entity"$$$$Darrell Hughes$$$$Person$$$$Darrell Hughes is the chief people officer of Ryanair, who announced the termination of Aidan Murray's employment.)
4. ("entity"$$$$Ryanair's chief people officer$$$$Job$$$$Ryanair's chief people officer is responsible for overseeing the airline's human resources department and ensuring a safe and secure work environment.)
5. ("entity"$$$$The Independent$$$$News Organization$$$$The Independent is a British online news publication that reported on the story of Aidan Murray's dismissal.)
6. ("entity"$$$$The Financial Times$$$$New

Here is the output based on the provided text:

**Entities**

("entity"$$$$KeyBank$$$$Company$$$$KeyBank is a bank-based financial services company that provides deposit, lending, cash management, and investment services to individuals and businesses in 15 states under the name KeyBank National Association through a network of approximately 1,000 branches and approximately 1,300 ATMs.)

("entity"$$$$American Fork Branch$$$$Branch$$$$The American Fork Branch is a financial institution branch of KeyBank, located in American Fork, Utah, that offers a range of financial services, including financial wellness center, mortgage, investments, business banking, and more.)

("entity"$$$$Five.12 Foundation$$$$Non-Profit Organization$$$$The Five.12 Foundation is a non-profit organization based in Alpine, Utah, that organizes and distributes weekend food bags to students in need at four American Fork elementary schools.)

("entity"$$$$Steve Jobs$$$$Person$$$$Steve Jobs was a co-founder of Apple Inc




Extracting paths from text: 100%|██████████| 50/50 [03:57<00:00,  4.75s/it]


Here are the identified entities and their entity types, along with the extracted relationships:

**Entities:**

("entity"$$$$Newcastle United$$$$Team$$$$Newcastle United is a professional football club based in Newcastle upon Tyne, England, that plays in the Premier League.)

("entity"$$$$Aston Villa$$$$Team$$$$Aston Villa is a professional football club based in Aston, Birmingham, England, that plays in the Premier League.)

("entity"$$$$Eddie Howe$$$$Person$$$$Eddie Howe is an English professional football manager and former player who is the current manager of Newcastle United.)

("entity"$$$$Unai Emery$$$$Person$$$$Unai Emery is a Spanish professional football manager who is the current manager of Aston Villa.)

("entity"$$$$Brentford$$$$Team$$$$Brentford is a professional football club based in Brentford, London, England, that plays in the Premier League.)

("entity"$$$$Man City$$$$Team$$$$Manchester City is a professional football club based in Manchester, England, that plays in




Generating embeddings: 0it [00:00, ?it/s]






Generating embeddings: 100%|██████████| 9/9 [00:00<00:00, 20.74it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [144]:
index.property_graph_store.get_triplets()

[[EntityNode(label='Company', embedding=None, properties={'id': 'Chevron', 'entity_description': 'Chevron Corporation is an American multinational energy corporation that is engaged in various businesses worldwide, including the production of crude oil and natural gas, refining, and the marketing of petroleum products and petrochemicals.', 'triplet_source_id': '2277f400-a654-483e-9c8d-6df52456046d'}, name='Chevron'),
  Relation(label='Operates In', source_id='Chevron', target_id='O&G', properties={'relationship_description': 'Chevron operates in the Oil and Gas (O&G', 'triplet_source_id': 'd9978e68-30f4-4f30-81f7-53910c43bc8a'}),
  EntityNode(label='Sector', embedding=None, properties={'id': 'O&G', 'entity_description': 'The Oil and Gas sector refers to the industry that extracts and refines crude oil, natural gas, and other petroleum products.', 'triplet_source_id': '2277f400-a654-483e-9c8d-6df52456046d'}, name='O&G')],
 [EntityNode(label='Company', embedding=None, properties={'id': '

In [154]:
print(len(index.property_graph_store.get_triplets()))

748


In [146]:
index.property_graph_store.get_triplets()[10][0].properties

{'id': 'Chevron',
 'entity_description': 'Chevron Corporation is an American multinational energy corporation that is engaged in various businesses worldwide, including the production of crude oil and natural gas, refining, and the marketing of petroleum products and petrochemicals.',
 'triplet_source_id': '2277f400-a654-483e-9c8d-6df52456046d'}

In [123]:
index.property_graph_store.get_triplets()[10][1].properties

{'relationship_description': 'Chevron has partnered with JHVEPhoto to obtain high-quality images and videos for its marketing and corporate communications efforts.',
 'triplet_source_id': '2277f400-a654-483e-9c8d-6df52456046d'}

### Build communities

This will create communities and summary for each community.

In [155]:
index.property_graph_store.build_communities()

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/text_generation/client.py:70: DeprecationWarning: `text_generation` clients are deprecated and will be removed in the near future. Please use the `InferenceClient` from the `huggingface_hub` package instead.
  warnings.warn(DEPRECATION_WARNING, DeprecationWarning)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/text_generation/client.py:553: DeprecationWarning: `text_generation` clients are deprecated and will be removed in the near future. Please use the `InferenceClient` from the `huggingface_hub` package instead.
  warnings.warn(DEPRECATION_WARNING, DeprecationWarning)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('T

### Create QueryEngine

In [156]:
query_engine = GraphRAGQueryEngine(
    graph_store=index.property_graph_store,
    llm=llm,
    index=index,
    similarity_top_k=10,
)

### Querying

In [157]:
response = query_engine.query(
    "What are the main news discussed in the document?"
)
display(Markdown(f"{response.response}"))

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; 

Here is a concise final response:

The main news discussed in the document are:

* The termination of Aidan Murray, the chief pilot of Ryanair, due to allegations of sexual misconduct.
* MDA Ltd.'s participation in the Jefferies Virtual Space Summit, including a fireside chat with Vito Culmone, Chief Financial Officer, and Shereen Zahawi, Senior Director, Investor Relations, as well as the release of a press release through PR Newswire.

In [158]:
response = query_engine.query("What are the main news in energy sector?")
display(Markdown(f"{response.response}"))

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; 

Here is the combined, concise response:

Based on the provided community summary, there is no specific news related to the energy sector mentioned. While Chevron, a major player in the Oil and Gas industry, is mentioned, the summary does not contain any specific news or updates related to the energy sector. However, it can be inferred that Chevron's Q2 consensus earnings estimates could be considered a potential news item in the energy sector. Additionally, its partnership with JHVEPhoto for high-quality visual content could also be related to the energy sector.

In [159]:
response = query_engine.query("What are the main news around sports?")
display(Markdown(f"{response.response}"))

/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/rbrugaro/miniforge3/envs/llamaindex/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; 

Here is the combined and concise response:

The main news around sports includes:

* Arsenal's rejection of Jude Bellingham's transfer bid and new league fixtures, as well as their rivalry with Chelsea and Manchester United in the Premier League.
* Chelsea's bid rejection for Mason Mount and targeting of him, André Onana, and Diogo Costa, as well as their rivalry with Manchester City and Manchester United.
* Manchester United's rejected bid for Harry Kane and targeting of Mason Mount, André Onana, and Diogo Costa, as well as their rivalry with Chelsea and Manchester City.
* The upcoming Premier League matches between several teams, including Arsenal vs. Manchester City, Chelsea vs. Manchester City, and Tottenham vs. Manchester City.
* The possibility of Real Madrid visiting Manchester City in the Champions League and Inter Milan visiting Manchester City in the Champions League.
* Manchester United's interest in signing Porto goalkeeper Diogo Costa and Chelsea's competition with Manchester United for the signing of André Onana.
* Newcastle United's upcoming Premier League matches, including their first match against Aston Villa and their final match against Brentford.
* The long-standing rivalries between various teams, including Liverpool's rivalry with Manchester City, Manchester United, and Newcastle United, and Manchester City's rivalry with Arsenal, Newcastle United, and Manchester United.
* The interest in signing transfer targets, including Jude Bellingham, Mason Mount, Harry Kane, and Diogo Costa.
* The upcoming matches in the Premier League, including Manchester City's matches against Arsenal, Chelsea, Manchester United, Brighton, and Tottenham.
* The possibility of Real Madrid and Inter Milan visiting Manchester City in the Champions League.
* The long-standing rivalries between various teams, including Burnley's rivalry with Liverpool and West Ham United's rivalry with Liverpool.

These news items highlight the competitive nature of the Premier League, the transfer activity surrounding top teams, and the intense rivalries between top clubs.